In [1]:
import pandas 

Retrieving business, training and testing reviews dataframe from pickle

In [24]:
business_df = pandas.read_pickle('business.pkl')
review_train_df = pandas.read_pickle('reviews_train.pkl')
review_test_df = pandas.read_pickle('reviews_test.pkl')

Calculating overall rating average and average of ratings by a particular user and for a particular business

In [25]:
overall_avg_rating = review_train_df["stars"].mean() #average of all the ratings in the dataset
avg_user_rating_df = review_train_df.groupby(["user_id"]).mean() #average of all the ratings by a particular user
avg_business_rating_df = review_train_df.groupby(["business_id"]).mean() #average of all the ratings for a particular business

Calculating rating by a particular user for a particular business by baseline approach

In [15]:
def calc_rating_baseline(user_id, business_id):
    user_bias = 0
    business_bias = 0
    user_rating = avg_user_rating_df[avg_user_rating_df.index == user_id]["stars"]
    if not user_rating.empty:
        user_bias = user_rating.item() - overall_avg_rating
    business_rating = avg_business_rating_df[avg_business_rating_df.index == business_id]["stars"]
    if not business_rating.empty:
        business_bias = business_rating.item() - overall_avg_rating
    return overall_avg_rating + user_bias + business_bias

Calculating user to business ratings for the held out test dataset

In [16]:
review_test_result_df = review_test_df.copy(deep = True)
review_test_result_df["stars"] = 0

In [17]:
review_test_result_df['stars'] = review_test_df.apply(lambda x: calc_rating_baseline(x[0], x[1]),axis = 1)

Save baseline results to pickle

In [19]:
review_test_result_df.to_pickle("./baseline_results.pkl")

Fetching unique number of categories in the dataset

In [20]:
unique_categories = business_df["categories"].str.split(", ").tolist()
unique_categories = [val for sublist in unique_categories if sublist for val in sublist] 
unique_categories = set(unique_categories)
len(unique_categories)

140

Calculating RMSE for the baseline results

In [21]:
def calc_RMSE():
    return ((review_test_result_df.stars - review_test_df.stars) ** 2).mean() ** 0.5
result = calc_RMSE()
result

1.1064904873251364